In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.1'
spark_version = 'spark-3.4.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 338 kB in 2s (156 kB/s)
Reading package lists... Done


In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
 # 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)

home_sales_df = spark.read.csv("file://" + SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

# Show the first few rows of the DataFrame
home_sales_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [ ]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView('home_sales')


In [ ]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
avg_price_query = """
SELECT YEAR(date) as year, ROUND(AVG(price), 2) as avg_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY YEAR(date)
ORDER BY YEAR(date)
"""

avg_price_results = spark.sql(avg_price_query)
avg_price_results.show()



+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [ ]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
avg_price_built_query = """
SELECT date_built, ROUND(AVG(price), 2) as avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
ORDER BY date_built
"""
spark.sql(avg_price_built_query).show()



+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [ ]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
avg_price_2_query = """
SELECT date_built, ROUND(AVG(price), 2) as avg_price
FROM home_sales
WHERE bedrooms = 3
    AND bathrooms = 3
    AND floors = 2
    AND sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built
"""
spark.sql(avg_price_2_query).show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [ ]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
# Record start time
start_time = time.time()

avg_price_view_query = """
SELECT view AS view_rating, ROUND(AVG(price), 2) as avg_price
FROM home_sales
WHERE price >= 350000
GROUP BY view
ORDER BY view
"""

#Execute query; calculate and print time elapsed
spark.sql(avg_price_view_query).show()
print("--- %s seconds ---" % (time.time() - start_time))

+-----------+---------+
|view_rating|avg_price|
+-----------+---------+
|          0|403848.51|
|          1|401044.25|
|          2|397389.25|
|          3| 398867.6|
|          4|399631.89|
|          5|401471.82|
|          6|395655.38|
|          7|403005.77|
|          8|398592.71|
|          9|401393.34|
|         10|401868.43|
|         11|399548.12|
|         12|401501.32|
|         13|398917.98|
|         14|398570.03|
|         15| 404673.3|
|         16|399586.53|
|         17|398474.49|
|         18|399332.91|
|         19|398953.17|
+-----------+---------+
only showing top 20 rows

--- 0.7611875534057617 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")


DataFrame[]

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [ ]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.
# Record start time
start_time = time.time()

query = """
    SELECT view AS view_rating, ROUND(AVG(price), 2) as avg_price
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view_rating
    ORDER BY view_rating
"""

spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))


+-----------+---------+
|view_rating|avg_price|
+-----------+---------+
|          0|403848.51|
|          1|401044.25|
|          2|397389.25|
|          3| 398867.6|
|          4|399631.89|
|          5|401471.82|
|          6|395655.38|
|          7|403005.77|
|          8|398592.71|
|          9|401393.34|
|         10|401868.43|
|         11|399548.12|
|         12|401501.32|
|         13|398917.98|
|         14|398570.03|
|         15| 404673.3|
|         16|399586.53|
|         17|398474.49|
|         18|399332.91|
|         19|398953.17|
+-----------+---------+
only showing top 20 rows

--- 0.645082950592041 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.mode("overwrite").partitionBy("date_built").parquet("home_sales_parquet_data")


# New Section

In [ ]:
# 11. Read the parquet formatted data.
home_sales_from_parquet = spark.read.parquet("home_sales_parquet_data")

In [ ]:
# 12. Create a temporary table for the parquet data.
home_sales_from_parquet.createOrReplaceTempView("home_sales_parquet_view")

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

query = """
    SELECT view AS view_rating, ROUND(AVG(price), 2) as avg_price
    FROM home_sales_parquet_view
    WHERE price >= 350000
    GROUP BY view_rating
    ORDER BY view_rating
"""
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+-----------+---------+
|view_rating|avg_price|
+-----------+---------+
|          0|403848.51|
|          1|401044.25|
|          2|397389.25|
|          3| 398867.6|
|          4|399631.89|
|          5|401471.82|
|          6|395655.38|
|          7|403005.77|
|          8|398592.71|
|          9|401393.34|
|         10|401868.43|
|         11|399548.12|
|         12|401501.32|
|         13|398917.98|
|         14|398570.03|
|         15| 404673.3|
|         16|399586.53|
|         17|398474.49|
|         18|399332.91|
|         19|398953.17|
+-----------+---------+
only showing top 20 rows

--- 1.1928176879882812 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales")


DataFrame[]

In [ ]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


False